#### Step 0

0. defining location
1. defining peak power and technology type
2. define TRACKING_TYPE (fixed, single axis, two axes)
3. define RACKING_MODEL
4. define TILT (used both for fixed modules and single axis tracking)
5. define AZIMUTH (used both for fixed modules and single axis tracking)
6. define the surface type

based on such information, the user should be able to:

7. choose the module
8. choose the inverter

1) Goals & site

- Location: Latitude/Longitude (map pin), auto-fill altitude (pvgis).
- Target peak power (kWp) and Max area (m²) (optional).
- Tech type (dropdown): TechChoice.
- Tracking type (dropdown): TrackingType.
- Mounting place (dropdown): RackingType
- Azimuth (dial/compass) & Tilt (slider).
- Surface type (dropdown → sets albedo): SurfaceType

First of all we are going to load the default data

In [1]:
from common.enums import PriceType
from models.gisrecs.aggregate_basic_info import AggregateBasicInfo
from models.gisrecs.energy_plant_edit_info_container import EnergyPlantEditInfoContainer
from models.gisrecs.pod_basic_info import PoDBasicInfo
from models.gisrecs.prices_search_info_container import TechnologyPriceDetail, TechnologyPriceInfoContainer


payload = EnergyPlantEditInfoContainer(
        project_uri=str('96051c0d-b960-439b-aa81-de9f24ea9c04'),
        latitude=round(float(41.9028), 6),
        longitude=round(float(12.4964), 6),
        selected_aggregate=AggregateBasicInfo(
            uri=str('db6d447f-7cf2-44d7-b44d-1bf31d7b7dd4'),
            name='test aggregate'
        ),
        selected_pod=PoDBasicInfo(
            uri=str('2eadcc55-f372-423a-9371-01adf19036e0'),
            code='IT00EXXXXXXXXXX',
            pod_type=1
            ),
        battery=None,
        existing_energy_plants=[
            *[
            ], 
            *[
            ]
        ],
        pv_uris_with_battery_connection=[
        ],
        prices = [
            TechnologyPriceInfoContainer(
                name='test pv price',
                price_type=PriceType.TECHNOLOGY_PRICE,
                price_status=0,
                count=1,
                price_details=[
                    TechnologyPriceDetail(
                        capex_price_kwh=250,
                        opex_price_kwh=15,
                        inverter_cost=2000,
                        technology_type=0,
                        minimum_power=0,
                        maximum_power=10000,
                    )
                ]
            )
        ],
        simulation_precision=1000
    )

The user will have to insert the mandatory fields

In [2]:
from apis.pvlib_helper.geographical_helper import GeographicalHelper
from common.enums import EnergyPlantType, RackingType, SurfaceType, TechChoice, TrackingType
from models.gisrecs.energy_plant_edit_info_container import RequestedPVSystemCharacteristics
from models.gisrecs.tech_components_criteria import TechComponentsCriteria


payload.plant_type = EnergyPlantType.PV
payload.requested_pv_system_characteristics.append(
    RequestedPVSystemCharacteristics(
        peak_power = 6,
        angle = 11.5,
        aspect = 0,
        square_meters=0
    )
)
payload.tracking_type = TrackingType.fixed
payload.mounting_place = RackingType.semi_integrated
payload.material = TechChoice.Mono_c_Si
payload.surface_type = SurfaceType.concrete
altitude = GeographicalHelper.get_elevation_opentopo(lat=payload.latitude, lon=payload.longitude)
payload.set_altitude(altitude)

criteria = TechComponentsCriteria(
    requested_dc_kwp=payload.total_peak_power,
    latitude=payload.latitude,
    longitude=payload.longitude,
    altitude=payload.altitude,
    material=payload.material,
    mounting_place=payload.mounting_place,
    surface_type=payload.surface_type
)

At this point we have all the elements to choose the pv module technology and/or the inverter technology.

In [3]:
from apis.pvlib_helper.geographical_helper import GeographicalHelper
from apis.pvlib_helper.pvsystems_helper import PVSystemHelper


pv_system_helper = PVSystemHelper()

u_c, u_v = RackingType.get_default_model_temperature_params(criteria.mounting_place)
hot_temperature, cold_temperature = GeographicalHelper.get_extreme_ambient_temperatures(
    lat=criteria.latitude, 
    lon=criteria.longitude, 
    altitude=criteria.altitude,
    surface_type=criteria.surface_type.name if payload.surface_type is not None else None,
    racking_type=criteria.mounting_place.name
    )
filtered_modules = pv_system_helper.get_cec_modules_by_tech_choice(TechChoice.get_tech_choice_descritpion(criteria.material))
filtered_modules = pv_system_helper.filter_legacy_or_out_of_scope_cec_modules(filtered_modules)
filtered_modules = pv_system_helper.filter_cec_modules_by_racking_type(filtered_modules, criteria.mounting_place)
filtered_inverters = pv_system_helper.get_cec_inverters_by_kwp(criteria.requested_dc_kwp)
filtered_modules, filtered_inverters = pv_system_helper.filter_cec_modules_by_voltage(
    filtered_modules.T,
    filtered_inverters,
    u_v=u_v,
    u_c=u_c,
    hot_temperature_ambient=hot_temperature,
    cold_temperature_ambient=cold_temperature
)

In [4]:
from models.simulatorrecs.pv_plant import InverterBasicInfo, ModuleBasicInfo, TechComponentEditInfoContainer

possible_tech_components = TechComponentEditInfoContainer(
    u_c=u_c,
    u_v=u_v,
    hot_ambient_temperature=hot_temperature,
    cold_ambient_temperature=cold_temperature,
    modules=[ModuleBasicInfo(name=name, nominal_power=m['STC']) for name, m in filtered_modules.T.iterrows()],
    inverters=[InverterBasicInfo(name=name, nominal_power=i['Paco']) for name, i in filtered_inverters.T.iterrows()]
)

##### User chooses the pv module

the app must, then, filter again the dropdown of the technology that the user has not picked yet. The procedure is the same for the two choices, since the method pv_system_helper.filter_cec_modules_by_voltage deals with it.

In [5]:
possible_tech_components.module = possible_tech_components.modules[0]

In [6]:
import pandas as pd

filtered_module = pd.DataFrame(filtered_modules[possible_tech_components.module.name])
u_c, u_v = RackingType.get_default_model_temperature_params(criteria.mounting_place)
filtered_modules, filtered_inverters = pv_system_helper.filter_cec_modules_by_voltage(
    filtered_module.T,
    filtered_inverters,
    u_v=u_v,
    u_c=u_c,
    hot_temperature_ambient=hot_temperature,
    cold_temperature_ambient=cold_temperature
)
possible_tech_components.modules = [ModuleBasicInfo(name=name, nominal_power=m['STC']) for name, m in filtered_modules.T.iterrows()]
possible_tech_components.inverters = [InverterBasicInfo(name=name, nominal_power=i['Paco']) for name, i in filtered_inverters.T.iterrows()]

In [7]:
possible_tech_components.inverter = possible_tech_components.inverters[0]

#### Step 1

0. choosing the number of modules in series (n_series)
1. choosing the number of series in parallel (n_strings)
2. suggest an ideal number of inverters
3. choose number of inverters
4. output the actual peak power achieved
5. the app must tell the user when the configuration he is forcing is not feasible

In [ ]:
from models.gisrecs.pv_cabling_scheme import PvCablingScheme


payload = PvCablingScheme(
    peak_power=payload.total_peak_power,
    latitude=payload.latitude,
    longitude=payload.longitude,
    altitude=payload.altitude,
    hot_ambient_temperature=possible_tech_components.hot_ambient_temperature,
    cold_ambient_temperature=possible_tech_components.cold_ambient_temperature,
    mounting_place=payload.mounting_place,
    module=possible_tech_components.module,
    inverter=possible_tech_components.inverter,
    u_c=possible_tech_components.u_c,
    u_v=possible_tech_components.u_v,
    gcr=possible_tech_components.gcr,
    b_0=possible_tech_components.b_0
)

AttributeError: 'EnergyPlantEditInfoContainer' object has no attribute 'peak_power'

In [9]:
module = payload.get_module_parameters
inverter = payload.get_inverter_parameters

In [10]:
temperature_model_parameters = payload.get_temperature_model_parameters
loc = payload.get_location

In [11]:
n_series, n_strings, sizing = pv_system_helper.pick_stringing(
    module, 
    inverter,
    t_cell_cold=PVSystemHelper._estimate_cell_T(payload.cold_ambient_temperature, Uc=payload.u_c, Uv=payload.u_v), 
    t_cell_hot=PVSystemHelper._estimate_cell_T(payload.hot_ambient_temperature, Uc=payload.u_c, Uv=payload.u_v), 
    dc_ac_ratio=1.20
)

In [12]:
print("Picked:", n_series, "modules per string,", n_strings, "strings")
print("Sanity:", sizing)

Picked: 7 modules per string, 1 strings
Sanity: {'mppt_low': 100.0, 'mppt_high': 416.0, 'vdcmax': 416.0, 'idcmax': 15.486629, 'paco': 4600.0, 'Vmp_cold': 33.49912269436517, 'Vmp_hot': 23.688159989112602, 'Voc_cold': 42.476276036440595, 'n_series_min_by_mppt': 5, 'n_series_max_by_mppt': 12, 'n_series_max_by_voc': 9, 'chosen_series': 7, 'Pmp_module_stc': 350.35, 'Pmp_string_stc': 2452.4500000000003, 'dc_target_W': 5520.0, 'chosen_strings': 1, 'total_imp_hot': 8.951241616758685, 'n_mppt': None, 'strings_per_mppt': None, 'per_mppt_cap': None}


In [13]:
payload.set_modules_per_string(n_series)
payload.set_strings(n_strings)
payload.set_number_of_inverters()

In [14]:
payload.number_of_inverters

2

In [15]:
payload.effective_peak_power_dc

4.9049000000000005

In [16]:
payload.effective_peak_power_ac

9.2

In [17]:
payload.dcac_ratio

0.5331413043478261

#### Step 2

4. define MAX_ANGLE if it is single axis tracking
5. define BACKTRACK if it is single axis tracking (True vs False)
6. define GCR if it is single axis tracking (default 0.35)
7. define temperature parameters (rules for default parameters is defined in class RackingType)
8. define b0 parameter of the module for IAM coefficient (defulat 0.05) for the ashrae aoi model
0. define wiring material (WiringMaterial enum)
1. define wiring length
2. define wiring cross section
3. define AC_WIRING_FACTOR
4. define AVAILABILITY
5. define LID_FACTOR
6. define MODULE_QUALITY_FACTOR
7. define MISMATCH_FACTOR
8. extract SOILING_MONTHLY (Monthly soiling transmittance)